# Optymalizacja zapytań django orm

Pracować będziemy na modelu postaci:

<code>class Product(models.Model):
    title = models.CharField(max_length=100)
    manufacturer = models.CharField(max_length=100)
    price = models.IntegerField()
</code>
<code>
    a = models.CharField(max_length=100)
    b = models.CharField(max_length=100)
    c = models.CharField(max_length=100)
</code>
<code>    
    product_secret_id = models.CharField(max_length=100)
</code>

Dopisz powyższy model do jednej ze swoich aplikacji.

Najpierw linijki, które pozwolą nam swobodnie korzystać z Django w notatniku Jupyter.

In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'intro.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

Zaimportujmy model

In [13]:
from orm_app.models import Product

In [3]:
from hello.models import Rower

Dodajmy kilka wpisów

In [9]:
rower1 = Rower.objects.create(
    type = "Men",
    number=5,
    price=35,
    id = '',
)

rower2 = Rower.objects.create(
    type = "Women",
    number=5,
    price=35,
    id = '',
)

rower3 = Rower.objects.create(
    type = "child",
    number=2,
    price=15,
    id = ''
)

ValueError: Field 'id' expected a number but got ''.

In [14]:
product1 = Product.objects.create(
    title='rower_men',
    manufacturer='romet',
    price=35,
    a='1.png',
    b='6',
    c='',
    product_secret_id='1111'
)

product2 = Product.objects.create(
    title='rower_women',
    manufacturer='romet',
    price=35,
    a='2.png',
    b='6',
    c='',
    product_secret_id='2222'
)

product3 = Product.objects.create(
    title='rower_kid',
    manufacturer='romet',
    price=35,
    a='3.png',
    b='2',
    c='',
    product_secret_id='3333'
)

I napiszmy zapytanie o wszystkie wpisy w tabelce.

In [11]:
products = Product.objects.all()
print(products.query)

NameError: name 'Product' is not defined

Powyższe zapytanie zwraca nam wartości WSZYSTKICH KOLUMN ze wszystkich wpisów w tabelce. Jak napisać zapytanie, które wyciągnie wartości tylko z wybranych kolumn?

Możemy oczywiście wyciągnąć z queryseta tylko te wartości, które nas interesują

In [15]:
[(product.title, product.manufacturer, product.price) for product in products]

NameError: name 'products' is not defined

Ale w ten sposób wciąż wyciągamy z bazy znacznie więcej informacji niż jest nam potrzebne (i dopiero w pythonie wybieramy z tego wyciąganiętego zbiory tylko te informacje, które nas interesują). Może to generować szereg poważnych problemów:
- im więcej danych chcemy wyciągnąć z bazy, tym dłużej użytkownicy będą czakać na wykonanie zapytania (oraz przesyłanie wyciągniętych danych) 
- w czasie kiedy wykonywane jest zapytanie na bazie inne zapytania muszą czekać, co ogranicza liczbę użytkowników jaką nasza aplikacja może płynnie obsłużyc
- obecnie, często w rozwiązaniach chmurowych za wykonanie zapytania płaci się proporcjonalnie do czasu wykonywania tego zapytania na bazie, co jeszcze bardziej podnosi rangę problemu

Oczywistym staje się potrzeba zminimalizowania ilość danych przesyłanych pomiędzy bazą a aplikacją. Chcemy wyciągnąć z bazy wyłącznie te informacje, które są nam potrzebne (co zawsze jest dobrą praktyką).

Mamy 3 metody, których możemy w tym celu użyć:
- `only`
- `values`
- `defer`

### only

In [16]:
products = Product.objects.all().only('title', 'manufacturer', 'price')
print(products.query)

SELECT "orm_app_product"."id", "orm_app_product"."title", "orm_app_product"."manufacturer", "orm_app_product"."price" FROM "orm_app_product"


In [18]:
print(products)

<QuerySet [<Product: Product object (1)>, <Product: Product object (2)>, <Product: Product object (3)>, <Product: Product object (4)>, <Product: Product object (5)>, <Product: Product object (6)>, <Product: Product object (7)>, <Product: Product object (8)>, <Product: Product object (9)>]>


W wyniku otrzymujemy tradycyjnego queryseta, którego wartości to obiekty klasy Product

Użycie `only` nie zabezpiecza nas jednak przed odpytywaniem bazy o pola, których nie wyciągneliśmy z bazy. W takim przypadku django orm po prostu drugi raz uderzy na baze i pobierze tę wartość.

In [19]:
products[3].a

'1.png'

W skrajnych przypadkach może to prowadzić do wykonania setek/tysięcy dodatkowych zapytań na bazę. Dlatego należy zwrócić szczególną uwagę na to, żeby w metodzie only umieścić wszystkie pola modelu, które zamierzamy użyć.

Do dyspozycji mamt też drugą metodą - `values`, która w niektórych scenariuszach moze okazać się lepszym rozwiązaniem.

### values

In [20]:
products = Product.objects.all().values('title', 'manufacturer', 'price')
print(products.query)

SELECT "orm_app_product"."title", "orm_app_product"."manufacturer", "orm_app_product"."price" FROM "orm_app_product"


In [11]:
print(products)

<QuerySet [{'title': 'test1', 'manufacturer': 'test', 'price': 100}, {'title': 'test2', 'manufacturer': 'test', 'price': 10}, {'title': 'test3', 'manufacturer': 'test', 'price': 1}, {'title': 'rower_men', 'manufacturer': 'romet', 'price': 35}, {'title': 'rower_women', 'manufacturer': 'romet', 'price': 35}, {'title': 'rower_kid', 'manufacturer': 'romet', 'price': 35}]>


Tym razem dostaliśmy queryset, ale słowników (a nie obiektów klasy Product). 

In [21]:
print(products[0])

{'title': 'test1', 'manufacturer': 'test', 'price': 100}


In [13]:
print(type(products[0]))

<class 'dict'>


Korzyści:

1. słowniki pythonowe zajmują mniej pamięci niż obiekty modelu
2. nie jesteśmy już w stanie z pojedynczego elementu tego queryseta wyciągnąć wartość pola, którego nie zawarliśmy w parametrach metody `values`. 

In [22]:
print(products[0]['title'])

test1


KeyError: 'defer'

Istnieje jeszcze trzecia metoda, której możemy tutaj użyć - `defer`

### defer

In [14]:
# TODO